# Carteira Recomendada utilizando Yahoo Finance

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from xgboost import XGBRegressor

# Dados
### Lista de ações
Para iniciar a construção do modelo responsável pela **recomendação da carteira de investimentos**, é necessário selecionar as **ações que serão o foco do estudo**. Além disso, foi escolhido o **índice de mercado** correspondente, que servirá como referência para o cálculo das métricas e para a comparação do desempenho da carteira definida com o desempenho geral do mercado.

In [ ]:
tickers = [
    "ABEV3.SA", "ALPA4.SA", "AMER3.SA", "ASAI3.SA", "AZUL4.SA", "B3SA3.SA",
    "BBAS3.SA", "BBDC3.SA", "BBDC4.SA", "BBSE3.SA", "BPAC11.SA", "BRAP4.SA",
    "BRFS3.SA", "BRKM5.SA", "BRML3.SA", "CASH3.SA", "CCRO3.SA", "CIEL3.SA",
    "CMIG4.SA", "COGN3.SA", "CPFE3.SA", "CPLE6.SA", "CRFB3.SA", "CSAN3.SA",
    "CSNA3.SA", "CYRE3.SA", "DXCO3.SA", "ELET3.SA", "ELET6.SA", "EMBR3.SA",
    "ENGI11.SA", "EQTL3.SA", "EZTC3.SA", "GGBR4.SA", "GOAU4.SA", "GOLL4.SA",
    "HAPV3.SA", "HYPE3.SA", "IGTI11.SA", "ITSA4.SA", "ITUB4.SA", "JBSS3.SA",
    "KLBN11.SA", "LREN3.SA", "MGLU3.SA", "MRFG3.SA", "MRVE3.SA", "MULT3.SA",
    "NTCO3.SA", "PETR3.SA", "PETR4.SA", "PRIO3.SA", "RADL3.SA", "RAIL3.SA",
    "RENT3.SA", "RRRP3.SA", "SANB11.SA", "SBSP3.SA", "SMTO3.SA", "SOMA3.SA",
    "SUZB3.SA", "TAEE11.SA", "TIMS3.SA", "UGPA3.SA", "USIM5.SA", "VALE3.SA",
    "VBBR3.SA", "VIIA3.SA", "VIVT3.SA", "WEGE3.SA", "YDUQ3.SA"
]

reference_index = "^BVSP"

### Coleta dos dados
Os dados utilizados neste estudo foram obtidos por meio da **API do Yahoo Finance**, que disponibiliza informações históricas sobre o desempenho de ações negociadas no mercado financeiro.  
A extração foi realizada para o período previamente definido, considerando as ações selecionadas para análise.

A API fornece, para cada ativo, as seguintes variáveis:

- **Open** – preço de abertura da ação no respectivo dia de negociação.  
- **High** – preço máximo atingido pela ação no período considerado.  
- **Low** – preço mínimo registrado no período.  
- **Close** – preço de fechamento da ação.  
- **Volume** – quantidade total de ações negociadas no período.

Esses dados constituem a base de entrada para as etapas subsequentes de processamento, análise e modelagem.

In [ ]:
start_date = "2022-01-01"
end_date = "2025-10-08"

raw_data = yf.download(tickers, start=start_date, end=end_date)
reference_index_data = yf.download(reference_index, start=start_date, end=end_date)

print(raw_data.head())
print(reference_index_data.head())

### Limpeza dos dados
Com base nos dados obtidos da API, realizamos a **limpeza dos ativos** que não tiveram as informações necessárias disponíveis ou que apresentaram **dados faltantes**.

In [ ]:
tickers_to_remove = [
    t for t in raw_data.columns.get_level_values(1).unique()
    if raw_data.xs(t, axis=1, level=1).isnull().any().any()
]

data = raw_data.drop(columns=tickers_to_remove, level=1)
tickers = [t for t in tickers if t not in tickers_to_remove]

print("Tickers removidos:", tickers_to_remove)

### Visualização dos dados
Com os dados revisados, é possível gerar uma **visualização gráfica** do valor das ações selecionadas ao longo do período definido para o estudo.  
Essa visualização permite observar como os preços das ações **variam ao longo do tempo**, refletindo os fenômenos do mercado nacional e internacional.

In [ ]:
plt.figure(figsize=(15,5))

for ticker in tickers:
    plt.plot(data.index, data["Close", ticker], label=ticker)

plt.xlabel("Date")
plt.ylabel("Closing Price [R$]")
plt.title("Stock Prices")
plt.legend(
    loc='upper left',
    bbox_to_anchor=(1, 1),
    fontsize='small',
    frameon=False,
    ncol=3,
    title='Stock Data'
)
plt.show()

### Cálculo das métricas (feature engineering)
Com os dados fornecidos pela API, realizamos a **extração das métricas** que caracterizam a performance de cada ativo no mercado de capitais. Foram consideradas métricas essenciais para a análise de ações, permitindo avaliar tanto o retorno quanto o risco associado a cada ativo ao longo do período analisado:

- **Retorno acumulado** – compara o preço de fechamento do último dia com o do primeiro, medindo o **ganho ou perda do ativo** no horizonte temporal considerado.  
- **Volatilidade** – representa o **desvio padrão dos retornos diários** ao longo do período. Quanto maior a volatilidade, **maior o risco associado** ao ativo.  
- **Drawdown** – mede a **maior queda do preço em relação a um pico anterior** dentro do período analisado, indicando a **exposição máxima a perdas**.  
- **Volume médio** – calcula a **média diária de negociação do ativo**, fornecendo uma noção da **liquidez do mercado**.  
- **Índice de correlação com referência** – avalia a **relação do desempenho da ação com um índice de mercado de referência**, permitindo identificar **quanto o ativo se movimenta em conjunto com o mercado**.  
- **Sharpe ratio** – combina retorno e risco ao dividir o **retorno acumulado pelo desvio padrão dos retornos diários**, fornecendo uma medida de **retorno ajustado pelo risco**.

In [ ]:
keys = ["ticker","cumulative_return_period","return_std_deviation","volume_avg","drawdown","sharpe_ratio","correlation_index"]
metrics = {key: [] for key in keys}

reference_index_return = reference_index_data[('Close', reference_index)].pct_change().dropna()
reference_index_return = reference_index_return.reset_index(drop=True)

for ticker in tickers:
    close_values = data[('Close', ticker)]
    close_values = close_values.reset_index(drop=True)
    volume = data[('Volume', ticker)]

    cumulative_return_period = (close_values.iloc[-1] / close_values.iloc[0]) - 1
    daily_return = close_values.pct_change().dropna()
    return_std_deviation = daily_return.std()
    volume_avg = volume.mean()
    drawdown = (close_values/close_values.cummax()-1).min()
    sharpe_ratio = cumulative_return_period / return_std_deviation if return_std_deviation > 0 else 0
    correlation_index = daily_return.corr(reference_index_return)

    metrics["ticker"].append(ticker)
    metrics["cumulative_return_period"].append(cumulative_return_period)
    metrics["return_std_deviation"].append(return_std_deviation)
    metrics["volume_avg"].append(volume_avg)
    metrics["drawdown"].append(drawdown)
    metrics["sharpe_ratio"].append(sharpe_ratio)
    metrics["correlation_index"].append(correlation_index)

df = pd.DataFrame(metrics)
print(df)

### Preparação dos Dados para o Modelo
Foram selecionadas as variáveis de entrada responsáveis por caracterizar o desempenho de cada ativo, enquanto o **Sharpe Ratio** foi definido como variável alvo.  
Em seguida, os dados foram **padronizados** utilizando o método *StandardScaler* e divididos em **conjuntos de treino e teste**, garantindo a adequada preparação para o processo de modelagem.

In [ ]:
X = df[["cumulative_return_period","return_std_deviation","drawdown","volume_avg","correlation_index"]]
y = df["sharpe_ratio"]

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.35, random_state=42)

### Otimização dos Hiperparâmetros

Foi realizada uma **busca em grade (Grid Search)** para otimizar os hiperparâmetros do modelo **XGBoost Regressor**.  
O processo utilizou validação cruzada com **5 folds** e a métrica de avaliação **R²**, visando identificar a combinação de parâmetros que proporcionasse o melhor desempenho preditivo do modelo.

In [ ]:
param_grid = {
    'n_estimators': [100,200,300,400,500],
    'max_depth': [3,5,7],
    'learning_rate': [0.01,0.05,0.1],
    'subsample': [0.7,0.8,1.0]
}

grid_search = GridSearchCV(estimator=XGBRegressor(random_state=42),
                           param_grid=param_grid,
                           cv=5, scoring='r2', n_jobs=-1)
grid_search.fit(X_scaled, y)
optimized_parameters = grid_search.best_params_
print(optimized_parameters)

### Treinamento do Modelo

Com os hiperparâmetros otimizados, foi treinado o modelo **XGBoost Regressor** utilizando o conjunto de dados de treino.  
O treinamento teve como objetivo ajustar o modelo para capturar as relações entre as métricas de desempenho dos ativos e o **Sharpe Ratio**.

In [ ]:
modelo = XGBRegressor(
    **optimized_parameters,
    colsample_bytree=0.9,
    random_state=42
)
modelo.fit(X_train, y_train)

### Avaliação do Modelo

O modelo treinado foi avaliado utilizando o conjunto de teste, com base nas métricas **R²** e **RMSE (Root Mean Squared Error)**.  
Essas métricas permitem quantificar o desempenho preditivo do modelo e a precisão das estimativas geradas para o **Sharpe Ratio**.


In [ ]:
y_pred = modelo.predict(X_test)
print(f"R²: {r2_score(y_test, y_pred):.3f}")
print(f"RMSE: {np.sqrt(mean_squared_error(y_test, y_pred)):.4f}")

### Geração da Recomendação de Carteira

O modelo foi utilizado para **gerar previsões do Sharpe Ratio** para todos os ativos.  
Em seguida, os ativos foram **ordenados** pelo valor predito, e as **5 principais ações** foram selecionadas como recomendação de investimento.  
A tabela abaixo apresenta essas ações, junto com suas métricas de desempenho.

In [ ]:
df["score_predito"] = modelo.predict(X_scaled)
df = df.sort_values(by=["score_predito","ticker"], ascending=[False,True]).reset_index(drop=True)
df_recomendadas = df.head(5)

print("\nAções recomendadas:")
print(df_recomendadas[["ticker","cumulative_return_period","return_std_deviation","drawdown","score_predito"]])